In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchsummary import summary
import csv
import numpy as np

In [3]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(2, 3)
        self.fc2 = nn.Linear(3, 3)
        self.fc3 = nn.Linear(3, 2)

    def forward(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x

net = Net()
device = torch.device("cuda") 
net = Net().to(device)
summary(net, input_size = (1,2))
#print(net)

loss_function = nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                 [-1, 1, 3]               9
            Linear-2                 [-1, 1, 3]              12
            Linear-3                 [-1, 1, 2]               8
Total params: 29
Trainable params: 29
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00
----------------------------------------------------------------


In [4]:
#dummy data
#trainset = (torch.rand(2,1, device="cuda"), torch.rand(2,1, device="cuda") ) #Xs and Ys
trainset = torch.tensor([[1,2],[3,4]]).to(device)
print(trainset)

tensor([[1, 2],
        [3, 4]], device='cuda:0')


In [5]:
Load_data = True
dataset_path = "../2NN_dataset.csv"
if(Load_data):
    trainset = []
    with open(dataset_path) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        for row in csv_reader:
            #feature, data_label = int(row[0:2]),row[2:4]
            feature, data_label = (int(row[0]),int(row[1])),(float(row[2]),float(row[3]))
            trainset.append([feature, data_label])
            #print([feature, data_label])


    #trainset = np.array(trainset)
    trainset = torch.tensor(trainset).to(device)
    #print(trainset[1])
    feature, data_label = trainset[0]
    print(feature, data_label)

tensor([[2.5600e+02, 1.5300e+02],
        [3.3497e-02, 9.7117e-03]], device='cuda:0')
tensor([256., 153.], device='cuda:0') tensor([0.0333, 0.0101], device='cuda:0')


In [6]:
train = True
epochs = 15
if (train):
    net.train()
    for epoch in range(epochs):
        for data in trainset:  
            feature, data_label = data #
            #print(data)
            #print(feature)
            #print(data_label)
            net.zero_grad()  
            output = net(feature.view(1,2))
            loss = loss_function(output, data_label) 
            loss.backward() 
            optimizer.step() 
        print(loss)


d:\Gilbert\Offline_Projects\191T_ML_Workspace\191T_py3\lib\site-packages\torch\nn\modules\loss.py:528: UserWarning: Using a target size (torch.Size([2])) that is different to the input size (torch.Size([1, 2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


tensor(0.0001, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(7.0133e-05, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(0.0069, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(0.0009, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(0.0020, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(0.0024, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(0.0015, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(0.0056, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(0.0009, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(0.0015, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(0.0005, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(8.5763e-05, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(0.0006, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(0.0009, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(0.0060, device='cuda:0', grad_fn=<MseLossBackward>)


In [7]:
#save network
save = True
save = False
model_save_path = "2nd_NN.pt"
if (save):
    torch.save(net, model_save_path)

In [ ]:
#test evaluate
